In [3]:
import pandas as pd

In [19]:
# Hàm gọi api here để lấy thông tin vị trí từ địa chỉ chi tiết
import requests

def get_location(address, api_key ):
    #api_key = "mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"

    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key}'
    
    try:
        # Gửi yêu cầu API
        response = requests.get(url)
        
        # Nếu API key đã hết hạn, thử API key tiếp theo trong danh sách

        if response.status_code == 429:
            return {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
        else: 
            data = response.json()    
            # Xử lý kết quả và trả về
            if data == {'items': []}: 
                list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "True" }
            else: 
                result = data['items'][0]
                position = result['position']
                lat = position.get('lat', '')
                long = position.get('lng', '')
                address_info = result['address']
                city = address_info.get('county', '')
                district = address_info.get('city', '')
                ward = address_info.get('district', '')
                street = address_info.get('street', '')
                houseNumber = address_info.get('houseNumber', '')
                list_address = {"lat": lat, "lng": long, "House_number": houseNumber, "Street": street, "Ward": ward, "District": district, "City": city, "progress": "True" }
    except:
           # Nếu thử xảy ra lỗi trả ra tập rỗng và progess là False đánh dấu những case chưa xử lí
        list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
           
    return list_address

# Hàm xử lí dữ liệu đầu vào là dataframe
def out_put(df, api_key):
    house_numbers = []
    streets = []
    wards = []
    districts = []
    cities = []
    progresses = []
    
    for address in df["address"]:
        address_info = get_location(address,api_key)
        house_numbers.append(address_info["House_number"])
        streets.append(address_info["Street"])
        wards.append(address_info["Ward"])
        districts.append(address_info["District"])
        cities.append(address_info["City"])
        progresses.append(address_info["progress"])
    
    df["House_number"] = house_numbers
    df["Street"] = streets
    df["Ward"] = wards
    df["District"] = districts
    df["City"] = cities
    df["progress"] = progresses

    return df

In [4]:
df_hcm = pd.read_excel("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ga7huvI3Qscl4CgNQ4uDkHFqABMuZGUrVtgO-tuWy6K3qQ7IAh2J-FYvdDA2R6nQoSN-_PnY5NbE/pub?output=xlsx")

In [11]:
df_hcm.head(5)

,potentialid,contract_city,contract_state,address,retailer_id,approval_date,expiry_date,cf_pot_nganh_hang
0,5461834,hồ chí minh,huyện cần giờ,"ấp an lộc, xã tam thôn hiệp, huyện cần giờ, tp...",380749,2019-01-15,2039-02-15,15
1,7077514,hồ chí minh,huyện cần giờ,"98 tam thôn hiệp, ấp an nghĩa, xã an thới đông...",1400106,2022-08-03,2023-07-28,Thức ăn chăn nuôi/Thủy hải sản
2,2187044,hồ chí minh,huyện cần giờ,195 đường tắc xuất-kp phong thạnh- cần thạnh-c...,250341,NaT,2022-12-26,3949
3,34867532,hồ chí minh,huyện cần giờ,"địa chỉ: tổ 3 ấp an nghĩa, xã an thới đông, hu...",1154575,2021-10-30,2024-03-01,11957
4,4628769,hồ chí minh,huyện cần giờ,"220a rừng sát, xã bình khánh, huyện cần giờ, t...",344106,2018-11-30,2020-01-10,61


In [12]:
df_test = df_hcm[df_hcm["contract_state"] ==  "quận 1"]

In [13]:
df_test.shape

(5262, 8)

In [14]:
# Calculate the number of groups
num_groups = len(df_test) // 1000

# Calculate the number of remaining rows
remaining_rows = len(df_test) % 1000

# Create a list to store the new dataframes
dfs = []

# Split the dataframe into groups of two rows each
for i in range(num_groups):
    start_index = i * 1000
    end_index = start_index + 1000
    df_group = df_test[start_index:end_index].copy()
    dfs.append(df_group)

# Add the remaining rows to the last dataframe
if remaining_rows > 0:
    last_df = df_test[-remaining_rows:].copy()
    dfs.append(last_df)

dfs

[      potentialid contract_city contract_state  \
 105      33719300   hồ chí minh         quận 1   
 106      39936997   hồ chí minh         quận 1   
 107       1867523   hồ chí minh         quận 1   
 108      45498993   hồ chí minh         quận 1   
 109       4904065   hồ chí minh         quận 1   
 ...           ...           ...            ...   
 1100     23744431   hồ chí minh         quận 1   
 1101     50987371   hồ chí minh         quận 1   
 1102       410815   hồ chí minh         quận 1   
 1103     52294132   hồ chí minh         quận 1   
 1104     56063369   hồ chí minh         quận 1   
 
                                                 address  retailer_id  \
 105        42/2 hồ hảo hớn, p. cô giang, quận 1, tp.hcm      1108742   
 106   68/3 nguyễn thái bình, p. nguyễn thái bình, q....    500032451   
 107                 72/10 trần đình xu, cô giang quận 1       212218   
 108   715 lê trọng tấn, kp2, bình hưng hòa, bình tân...    500045953   
 109   4c trần hưng đ

In [18]:
num_elements = len(dfs)
print(num_elements)


6


In [22]:
list_api_keys = ["1EotnhALO_Xf7d5mlOVefECdfpbgPG_Dik0TeOcQauQ",
"92hxCAU523jWUFRXMhM-p3w66P3PxYfY_tZm9pPeShU",
"KwvOafjkobVhF8Iz63lMyNooS-hjaYgmHdZrqVVVHWw",
"N5qb2q5dSfly3fnqWMsDPqVMWUd1yRbTBxbT34hvw_g",
"SsQNhtNfemZJr_FzCYZlub29VeTTjymT3Qm6syC50W8",
"br4m9W3KMhrPa51Vru5M1mWJYCfaVFvWiP3kB2gEQUA",
"spdVYGb-b6cgokZBOuwGG4b3WbmgsxQqv7tudjdOfLc",
"0zeR-GEwo5Od-poIZYTGrw-I80fC2C0b0q4F1Boa8oc",
"UfHD68dH4O9ld-NS22Se6Bls98P9aPZhFq4hA_H7-74",
"c6NrKs_BpAX4OdRFKAVSDeUaQTjSpNF8dCxjcLT6AfE",
"WCpDyLiAACoL9zwSFQQa_HboP-HdWXSM601W1N02aeg",
"faJrm5egL12NmLivrDsbqz44I47DhbJQUIg89L3TdIo",
"cdXQHZ3zzsZv-n04kjimcml7e3BRKcR2H3wl8XPi6SU",
"RBRLSkMgpwy6QF0jKsXs-jXEliqtAwwD_jd0DbHrEN8",
"Dd-n66bTKOIKeQ0H6nRvIOMWatnJ_viPxKMNGpxsFa4",
"q-Ao_RRfZNyfQPp2XsnYGbrOItF6SlXN0jObPIKVnk",
"ufLjrWIrvjyLHzL1TZIEOZjKpgolDMBUPXUwoIblgs0",
"tvjh1g0Xygp6DMsH8n5hX41jgbNh3VE61MsnWHYkfZw",
"d_8_FhktHjeMh91gB9bBKGHgMJFC6auZqHThhlHL9Hw",
"P1RAwTjkXvZUQLjHyi06Zm1YP1KeaZMJdjaNN1ZeOeE",
"DQV3KQ2U9DLoro0jMQUoIr-PQMKBpKIEpOqXzF1A498",
"6IcCytBh4PkKOC3YE2Smw4IGdAyTD7XTtsmgtxF-kdY",
"PgxoArILmRDFQreVp3pC6bexYfcKIVwMMN4oKpa8VOo",
"mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"]

In [21]:
for i , api_key in zip(range(5),list_api_keys):
    print(i, api_key)

0 1EotnhALO_Xf7d5mlOVefECdfpbgPG_Dik0TeOcQauQ
1 92hxCAU523jWUFRXMhM-p3w66P3PxYfY_tZm9pPeShU
2 KwvOafjkobVhF8Iz63lMyNooS-hjaYgmHdZrqVVVHWw
3 N5qb2q5dSfly3fnqWMsDPqVMWUd1yRbTBxbT34hvw_g
4 SsQNhtNfemZJr_FzCYZlub29VeTTjymT3Qm6syC50W8


In [23]:
# tiến hành chạy multi thread
import threading
import time

final = []
def out_put_with_time(df, api_key):
    start_time = time.time()
    out_put(df, api_key)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Time: {execution_time/60} phút")

for df , api_key in zip(dfs, list_api_keys):
    thread = threading.Thread(target=out_put_with_time, args=(df,api_key))
    thread.start()
    thread.join()
    final.append(df)

print("Done")

Time: 6.1729407787323 phút
Time: 6.024663905302684 phút
Time: 46.587092034022014 phút
Time: 5.7858157078425085 phút
Time: 5.645662844181061 phút
Time: 1.582927926381429 phút
Done


In [26]:
merged_df = pd.concat(final, ignore_index=True)

In [27]:
merged_df

,potentialid,contract_city,contract_state,address,retailer_id,approval_date,expiry_date,cf_pot_nganh_hang,House_number,Street,Ward,District,City,progress
0,33719300,hồ chí minh,quận 1,"42/2 hồ hảo hớn, p. cô giang, quận 1, tp.hcm",1108742,2021-09-30,2025-12-30,9,42/2,Hẻm 42 Hồ Hảo Hớn,Phường Cô Giang,Quận 1,Hồ Chí Minh,True
1,39936997,hồ chí minh,quận 1,"68/3 nguyễn thái bình, p. nguyễn thái bình, q....",500032451,2022-12-22,2025-06-19,3949,68/3,Hẻm 68 Nguyễn Thái Bình,Phường Nguyễn Thái Bình,Quận 1,Hồ Chí Minh,True
2,1867523,hồ chí minh,quận 1,"72/10 trần đình xu, cô giang quận 1",212218,NaT,2027-11-15,77,72/1,Đường Trần Đình Xu,Phường Cô Giang,Quận 1,Hồ Chí Minh,True
3,45498993,hồ chí minh,quận 1,"715 lê trọng tấn, kp2, bình hưng hòa, bình tân...",500045953,2022-12-30,2024-12-30,3472,715,Đường Lê Trọng Tấn,Phường Bình Hưng Hòa,Quận Bình Tân,Hồ Chí Minh,True
4,4904065,hồ chí minh,quận 1,"4c trần hưng đạo, phạm ngũ lão, quận 1, hồ chí...",1160022,2021-11-16,2023-11-12,77,4C,Đường Trần Hưng Đạo,Phường Phạm Ngũ Lão,Quận 1,Hồ Chí Minh,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5257,28895930,hồ chí minh,quận 1,"121/13 bùi viện , phường phạm ngũ lão ,quận1",941486,2021-06-03,2022-07-02,3949,121/15,Hẻm 121 Bùi Viện,Phường Phạm Ngũ Lão,Quận 1,Hồ Chí Minh,True
5258,54791775,hồ chí minh,quận 1,"chung cư hà đô centrosa, 200 đường 3/2, p.12, ...",500348027,2023-10-09,2024-10-06,77,200,Đường 3 Tháng 2,Phường 12,Quận 10,Hồ Chí Minh,True
5259,20463342,hồ chí minh,quận 1,"địa chỉ: 4-17-28 kamiji, higashinari-ku, osaka",838847,2020-12-30,2022-12-29,3949,,,,,,True
5260,42834844,hồ chí minh,quận 1,"123a dương văn dương, phường tân qúy, quận tân...",1421114,2022-09-08,2023-09-10,73,123,Đường Dương Văn Dương,Phường Tân Quý,Quận Tân Phú,Hồ Chí Minh,True


In [28]:
merged_df[merged_df["progress"] == "False"]

,potentialid,contract_city,contract_state,address,retailer_id,approval_date,expiry_date,cf_pot_nganh_hang,House_number,Street,Ward,District,City,progress
1726,43639192,hồ chí minh,quận 1,"#15h borei sok san,prey pring thbong 2 ,sangka...",1506557,2022-10-19,2025-01-27,61,,,,,,False
2226,17069747,hồ chí minh,quận 1,"st.1 #494, sangkat kbal koh , khan chbar ampov...",404615,2020-10-24,2024-09-05,71,,,,,,False
2622,23534379,hồ chí minh,quận 1,"#380, st87bt , k. mean chay, phnom penh, campu...",874389,2021-02-19,2023-05-28,15,,,,,,False
4712,40822973,hồ chí minh,quận 1,"#405, khách sạn nikko saigon, 235 nguyễn văn c...",1357715,2022-06-24,2023-06-20,77,,,,,,False


In [29]:
merged_df.to_excel("hcm_quan1.xlsx", index=False)

In [30]:
merged_df.to_csv("hcm_quan1.csv", index=False)